In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style(style='darkgrid')
plt.style.use('ggplot')
%matplotlib inline

ModuleNotFoundError: No module named 'google'

In [ ]:
 #files.upload()
asthma = pd.read_csv('data/asthma.csv')

 #files.upload()
#PM2_5 = pd.read_csv('Daily_Census_Tract-Level_PM2.5_Concentrations__2011-2014_4.csv')

# files.upload()
states = pd.read_csv('states.csv')
state_fips_code = pd.read_csv('https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv')

In [ ]:
asthma_states = asthma.merge(states, left_on='LocationAbbr', right_on='State Code')
asthma_states.head()

In [ ]:
aap = asthma_states.query(
        'Question == "Current asthma prevalence among adults aged >= 18 years"' + 
        '& StratificationCategory1 == "Overall"' +
        '& DataValueType == "Age-adjusted Prevalence"' # Asthma prevalence is expressed as a percentage of the overall population
    )[['YearStart', 'State Code', 'Division', 'DataValue']].rename(
        columns={'YearStart': 'year',
                 'State Code': 'state',
                 'Division': 'div',
                 'DataValue': 'aap'}
    ).reset_index().drop(columns='index')
aap

In [ ]:
aap['aap'].isna().sum()
aap = aap.dropna()


If we want to run t-test, we need to be sure that the aap scores are approximately normally distrubuted for each region.

---



In [ ]:
for div in aap["div"].value_counts().index:
  plt.figure(div)
  plt.hist(aap[aap["div"]==div]["aap"], density= 1, color = 'c');
  plt.xlabel("aap dist")
 # plt.ylabel("density")
  plt.title(div);

 those are some relatively normal looking distributions that do not conation obvious outliers, we'll do QQ-plots to be certain

In [ ]:
import statsmodels.api as sm
import pylab as py
for div in aap["div"].value_counts().index:
  plt.figure(div)
  #plt.hist(, density= 1, color = 'c');
  sm.qqplot(aap[aap["div"]==div]["aap"], line = "45", fit = True);
  plt.xlabel("aap dist")
 # plt.ylabel("density")
  plt.title(div);

In [ ]:
aap["div"].value_counts().index


In [ ]:
overall_mean = np.mean(aap["aap"])

division_means = aap.groupby("div").agg(np.mean)
division_counts = aap.groupby("div").count()

division_means["count"] = division_counts["aap"]
division_means

In [ ]:

prevs = np.array(aap["aap"])
p_values_above = []
p_values_below = []
#We want to do some bootstrapping for every division
for div in aap["div"].value_counts().index:
  size = division_means.loc[div,"count"]
  actual_mean = division_means.loc[div,"aap"]
  boot_means = []
  #bootstrap step
  for _ in np.arange(200):
    boot = np.random.choice(prevs, replace = True, size = size )
    boot_means.append(np.mean(boot))

  #save the p-values

  #suppose null-the given region does not have more asthma prevelence than the general populace
  pval_above = sum(boot_means>actual_mean)/200
  p_values_above.append(pval_above)

  #suppose null-the given region does not have lower asthma prevelence than the general populace
  pval_below = sum(boot_means<actual_mean)/200
  p_values_below.append(pval_below)

  #plot our findings
  plt.figure(div)
  plt.hist(boot_means, density= 1, color = 'c');
  plt.xlabel("bootstrapped means")
  plt.ylabel("density")
  plt.title(div);
  plt.axvline(actual_mean, color="crimson");
  plt.savefig(div)

In [ ]:
pval_df = pd.DataFrame(aap["div"].value_counts().index)

pval_df["p_vals_above"] = p_values_above
pval_df["p_vals_below"] = p_values_below

In [ ]:
pval_df

In [ ]:
#TODO -- Bonferroni or B-H for Group-wise error rates
#borrowed from lab01 **B-H requires null p-vals to be indep


alpha = 0.05
def benjamini_hochberg(p_values, alpha):
    """
    Returns decisions on p-values using Benjamini-Hochberg.
    
    Inputs:
        p_values: array of p-values
        alpha: desired FDR (FDR = E[# false positives / # positives])
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    n = len(p_values)
    K = np.arange(n)
    p_values_copy = p_values.copy()
    p_values_copy.sort()
    
    opt_p = 0
    
    for k in K:
        if p_values_copy[k] <= [(k+1)*alpha/n]:
            
            opt_p = p_values_copy[k]
   

    decisions = p_values <= opt_p
    return decisions

above_decisions = benjamini_hochberg(p_values_above, alpha)
below_decisions = benjamini_hochberg(p_values_below, alpha)

pval_df["B-H reject null(is above)"] = above_decisions
pval_df["B-H reject null(is below)"] = below_decisions


In [ ]:
#Bonferroni also from lab01
def bonferroni(p_values, alpha_total):
    """
    Returns decisions on p-values using the Bonferroni correction.
    
    Inputs:
        p_values: array of p-values
        alpha_total: desired family-wise error rate (FWER = P(at least one false discovery))
    
    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    m = len(p_values)
    decisions = p_values <= (alpha_total/m)
    return decisions

    above_decisions = bonferroni(np.array(p_values_above), alpha)
below_decisions = bonferroni(np.array(p_values_below), alpha)

pval_df["Bonferroni reject null(is above)"] = above_decisions
pval_df["Bonferroni reject null(is below)"] = below_decisions

In [ ]:
pval_df